In [1]:
using Pkg
# cd("D:\\Home\\Git\\Oceananigans.jl")
# cd("C:\\Users\\Ali\\Documents\\Git\\Oceananigans.jl\\")
cd("/home/gridsan/aramadhan/Oceananigans.jl/")
Pkg.activate(".");

In [2]:
using Oceananigans

In [3]:
Nx, Ny, Nz = 100, 100, 50
Lx, Ly, Lz = 2000, 2000, 1000
model = Model((Nx, Ny, Nz), (Lx, Ly, Lz));

Planning Fourier transforms... (planner_flag=FFTW.PATIENT)
FFT!:    0.810180 seconds (44 allocations: 3.031 KiB)
IFFT!:   0.833096 seconds (16.84 k allocations: 733.125 KiB)
DCT!:    0.189930 seconds (11.51 k allocations: 596.992 KiB)
IDCT!:   0.184277 seconds (10.94 k allocations: 560.945 KiB)


In [ ]:
using Serialization

In [ ]:
model.ssp = nothing
@time serialize("model.jlser", model)

In [ ]:
filesize("model.jlser") / 1024^2

In [ ]:
@time model2 = deserialize("model.jlser");  # This seg faults when reading the FFTW plans.

In [4]:
import JLD

In [6]:
model.ssp = nothing
@time begin
    f = JLD.jldopen("model3.jld", "w")
    JLD.@write f model
    close(f)
end

  8.488431 seconds (29.10 M allocations: 1.331 GiB, 6.69% gc time)


In [7]:
filesize("model3.jld") / 1024^2

190.9367551803589

In [8]:
@time begin
    f = JLD.jldopen("model_compressed.jld", "w", compress=true)
    JLD.@write f model
    close(f)
end

  0.346759 seconds (2.11 M allocations: 67.080 MiB, 5.63% gc time)


In [9]:
filesize("model_compressed.jld") / 1024^2

2.0384092330932617

In [11]:
@time begin
    f = JLD.jldopen("model_compressed.jld", "r")
    model3 = read(f, "model");
    close(f)
end

  0.365751 seconds (12.15 k allocations: 191.317 MiB, 53.48% gc time)


In [21]:
using NetCDF

In [37]:
xC = collect(model.grid.xC)
yC = collect(model.grid.yC)
zC = collect(model.grid.zC)
xF = collect(model.grid.xF)
yF = collect(model.grid.yF)
zF = collect(model.grid.zF)

xC_attr = Dict("longname" => "Locations of the cell centers in the x-direction.", "units" => "m")
yC_attr = Dict("longname" => "Locations of the cell centers in the y-direction.", "units" => "m")
zC_attr = Dict("longname" => "Locations of the cell centers in the z-direction.", "units" => "m")
xF_attr = Dict("longname" => "Locations of the cell faces in the x-direction.", "units" => "m")
yF_attr = Dict("longname" => "Locations of the cell faces in the y-direction.", "units" => "m")
zF_attr = Dict("longname" => "Locations of the cell faces in the z-direction.", "units" => "m")

T_attr = Dict("longname" => "Temperature", "units" => "K")
u_attr = Dict("longname" => "Velocity in the x-direction", "units" => "m/s")

nc_filename = "deep_convection.nc"
isfile(nc_filename) && rm(nc_filename)

nccreate(nc_filename, "T", "xC", xC, xC_attr, "yC", yC, yC_attr, "zC", zC, zC_attr, atts=T_attr, compress=5)
ncwrite(model.tracers.T.data, nc_filename, "T")

nccreate(nc_filename, "u", "xF", xF, xF_attr, "yC", yC, yC_attr, "zC", zC, zC_attr, atts=u_attr, compress=5)
ncwrite(model.velocities.u.data, nc_filename, "u")

ncclose(nc_filename)

262144

In [38]:
ncinfo(nc_filename)


##### NetCDF File #####

/home/gridsan/aramadhan/Oceananigans.jl/deep_convection.nc

##### Dimensions #####

Name                                                Length                    
--------------------------------------------------------------------------------
zC                                                  50                        
xC                                                  100                       
xF                                                  101                       
yC                                                  100                       

##### Variables #####

Name                            Type            Dimensions                      

##### Attributes #####

Variable            Name                Value                                   
--------------------------------------------------------------------------------
zC                  units               m                                       
zC                  longname            L

--------------------------------------------------------------------------------
zC                              DOUBLE          zC                              
xC                              DOUBLE          xC                              
T                               DOUBLE          xC yC zC                        
u                               DOUBLE          xF yC zC                        
xF                              DOUBLE          xF                              
yC                              DOUBLE          yC                              


In [34]:
xC = collect(model.grid.xC)
yC = collect(model.grid.yC)
zC = collect(model.grid.zC)
xF = collect(model.grid.xF)
yF = collect(model.grid.yF)
zF = collect(model.grid.zF)

xC_attr = Dict("longname" => "Locations of the cell centers in the x-direction.", "units" => "meters")
yC_attr = Dict("longname" => "Locations of the cell centers in the y-direction.", "units" => "meters")
zC_attr = Dict("longname" => "Locations of the cell centers in the z-direction.", "units" => "meters")
xF_attr = Dict("longname" => "Locations of the cell faces in the x-direction.", "units" => "meters")
yF_attr = Dict("longname" => "Locations of the cell faces in the y-direction.", "units" => "meters")
zF_attr = Dict("longname" => "Locations of the cell faces in the z-direction.", "units" => "meters")

T_attr = Dict("longname" => "Temperature", "units" => "degrees Kelvin")

xC_dim = NcDim("xC", xC, xC_attr)
yC_dim = NcDim("yC", yC, yC_attr)
zC_dim = NcDim("zC", zC, zC_attr)
xF_dim = NcDim("xF", xF, xF_attr)
yF_dim = NcDim("yF", yF, yF_attr)
zF_dim = NcDim("zF", zF, zF_attr)

T_var = NcVar("T", [xC_dim, yC_dim, zC_dim], T_attr, eltype(model.tracers.T.data))

nc_filename = "deep_convection.nc"
isfile(nc_filename) && rm(nc_filename)

nc = NetCDF.create("deep_convection_low.nc", T_var)
NetCDF.putvar(nc, "T", model.tracers.T.data)
NetCDF.close(nc)

InexactError: InexactError: trunc(Int32, 8589934592)

In [28]:
eltype(model.tracers.T.data)

Float64